In [2]:
import torch
import torch.nn as nn
from torch.nn import functional
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F

import pandas as pd
import numpy as np
import time
import random
import os

os.chdir('../')

%load_ext autoreload
%autoreload 2

In [3]:
from model.encoder import Encoder
from model.decoder_v2 import Decoder_luong

from src.utils import time_since
from src.data import prepare_data
from src.data_loader import get_loader

In [4]:
dir_files = 'data/disambiguation/'
dir_test = os.path.join(dir_files, 'test')

In [5]:
for name_dir in ['all_eq', 'all_less']:
    dir_train = os.path.join(dir_files, name_dir)
    for name_file in ['all_f1_lemma', 'all_f1_random_lemma', 'all_meteor_lemma', 'all_meteor_random_lemma']:
        _, _, pairs_train, pairs_test, _= prepare_data(name_file, 'verbs_selected_lemma',
                                                                       max_length=100, dir_train=dir_train, 
                                                                       dir_test=dir_test)
        
        pd.DataFrame(pairs_train).to_csv(os.path.join(dir_train, name_file + '.tsv'), header=False, index=False, sep='\t')
                

Reading lines...
Read 40870 train pairs
Reading lines...


FileNotFoundError: [Errno 2] No such file or directory: 'None/dev_in.txt'

In [5]:
pd.DataFrame(pairs_test).to_csv(os.path.join(dir_test, 'verbs_selected_lemma.tsv'), header=False, index=False, sep='\t')

In [8]:
import itertools
from typing import Iterator, List, Dict
import fire
import torch
import torch.optim as optim
from allennlp.data.dataset_readers.seq2seq import Seq2SeqDatasetReader
from allennlp.data.iterators import BucketIterator
from allennlp.data.fields import TextField, IndexField
from allennlp.data.token_indexers import TokenIndexer, SingleIdTokenIndexer
from allennlp.data.tokenizers import Token
from allennlp.data import Instance

from allennlp.data.tokenizers.character_tokenizer import CharacterTokenizer
from allennlp.data.tokenizers.word_tokenizer import WordTokenizer
from allennlp.data.vocabulary import Vocabulary
from allennlp.nn.activations import Activation
from allennlp.models.encoder_decoders.simple_seq2seq import SimpleSeq2Seq
from allennlp.models import DecomposableAttention
from allennlp.modules.attention import LinearAttention, BilinearAttention, DotProductAttention
from allennlp.modules.seq2seq_encoders import PytorchSeq2SeqWrapper, StackedSelfAttentionEncoder
from allennlp.modules.text_field_embedders import BasicTextFieldEmbedder
from allennlp.modules.token_embedders import Embedding
from allennlp.predictors import SimpleSeq2SeqPredictor
from allennlp.training.trainer import Trainer
from allennlp.data.dataset_readers import DatasetReader

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [7]:
!pip install allennlp

    100% |████████████████████████████████| 5.6MB 1.5MB/s ta 0:00:011
  Using cached https://files.pythonhosted.org/packages/8f/86/df789c5834f15ae1ca53a8d4c1fc4788676c2e32112f6a786f2625d9c6e6/ftfy-5.5.1-py3-none-any.whl
    100% |████████████████████████████████| 51kB 13.0MB/s ta 0:00:01
    100% |████████████████████████████████| 1.5MB 739kB/s ta 0:00:011
    100% |████████████████████████████████| 552kB 828kB/s ta 0:00:011
  Using cached https://files.pythonhosted.org/packages/d4/2c/856344d9b69baf5b374c395b4286626181a80f0c2b2f704914d18a1cea47/conllu-0.11-py2.py3-none-any.whl
    100% |████████████████████████████████| 133kB 248kB/s ta 0:00:01
  Using cached https://files.pythonhosted.org/packages/5d/3c/d5fa084dd3a82ffc645aba78c417e6072ff48552e3301b1fa3bd711e03d4/pytorch_pretrained_bert-0.6.1-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/02/42/cca66659a786567c8af98587d66d75e7d2b6e65662f8daab75db708ac35b/flaky-3.5.3-py2.py3-none-any.whl
    100% |█████████████

  Using cached https://files.pythonhosted.org/packages/e1/ae/baedc9cb175552e95f3395c43055a6a5e125ae4d48a1d7a924baca83e92e/rsa-3.4.2-py2.py3-none-any.whl
    100% |████████████████████████████████| 5.4MB 695kB/s ta 0:00:01
  Using cached https://files.pythonhosted.org/packages/e6/35/f187bdf23be87092bd0f1200d43d23076cee4d0dec109f195173fd3ebc79/mock-2.0.0-py2.py3-none-any.whl
    100% |████████████████████████████████| 143kB 12.6MB/s ta 0:00:01
  Using cached https://files.pythonhosted.org/packages/a4/a5/da7887285564f9e0ae5cd25a453cca36e2cd43d8ccc9effde260b4d80904/aws_xray_sdk-0.95-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/c5/e1/1523fb1dab744e2c6b1f02446f2139a78726c18c062a8ddd53875abb20f8/pyaml-18.11.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/bf/5c/5fa238c0c5b0656994b52721dd8b1d7bf52ebd8786518dde794f44de86b6/python_jose-2.0.2-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/bd/5f/13e28a2f9abeda

    100% |████████████████████████████████| 204kB 795kB/s ta 0:00:01
  Using cached https://files.pythonhosted.org/packages/dc/12/8c44eabb501e2bc0aec0dd152b328074d98a50968d3a02be28f6037f0c6a/jsonpickle-1.1-py2.py3-none-any.whl
    100% |████████████████████████████████| 9.7MB 2.0MB/s ta 0:00:011
  Using cached https://files.pythonhosted.org/packages/63/f4/73669d51825516ce8c43b816c0a6b64cd6eb71d08b99820c00792cb42222/ecdsa-0.13-py2.py3-none-any.whl
  Running setup.py bdist_wheel for parsimonious ... done
  Stored in directory: /home/krivas/.cache/pip/wheels/b7/8d/e7/a0e74217da5caeb3c1c7689639b6d28ddbf9985b840bc96a9a
  Running setup.py bdist_wheel for word2number ... done
  Stored in directory: /home/krivas/.cache/pip/wheels/46/2f/53/5f5c1d275492f2fce1cdab9a9bb12d49286dead829a4078e0e
  Running setup.py bdist_wheel for jsonnet ... done
  Stored in directory: /home/krivas/.cache/pip/wheels/f0/47/51/a178b15274ed0db775a1ae9c799ce31e511609c3ab75a7dec5
  Running setup.py bdist_wheel for overrid